In [ ]:
def play_game(p1, p2, board, draw=False):
    
    currentPlayer = None
    while board.gameNotOver():
        if currentPlayer == p1:
            currentPlayer = p2
        else:
            currentPlayer = p1
        
        if draw:
            if draw == 1 and currentPlayer == p1:
                board.drawBoard()
            if draw == 2 and currentPlayer == p2:
                board.drawBoard()
            
        currentPlayer.takeAction(board)
        
        state = board.getState()
        #update the state in which the board is for both the players
        p1.updateStateHistory(state)
        p2.updateStateHistory(state)
        
        if draw:
            board.drawBoard()
        
        #update the value functions
        p1.updateEnv(board)
        p2.updateEnv(board)